In [1]:
%matplotlib inline

#import comet_ml
from comet_ml import Experiment
from comet_ml import Optimizer
API_KEY = os.environ['COMET_API_KEY']

# import keras
import keras
from keras import backend as K 

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu
from keras_retinanet.losses import smooth_l1, focal
from keras_retinanet.preprocessing import csv_generator
from keras_retinanet.utils.eval import evaluate

# other imports
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

setup_gpu(1)

Using TensorFlow backend.


2 Physical GPUs, 1 Logical GPUs


In [2]:
default_params = {
    'N': 3,
    'M': 10
}
dataset_path = Path('/data/students_home/fschipani/thesis/MSc-Thesis-PJ/Dataset/KAIST_MPD/fine_tuning_kaist_cars/ds')
train_annotations = dataset_path.joinpath('train_no_people.csv')
test_annotations = dataset_path.joinpath('test_w_people.csv')
classes = Path('/data/students_home/fschipani/thesis/MSc-Thesis-PJ/Dataset/KAIST_MPD/class_name_to_ID_CARS.csv')

def create_train_generator(N, M):
    train_generator = csv_generator.CSVGenerator(
        train_annotations,
        classes,
        transform_generator=None,
        visual_effect_generator=None,
        image_min_side=800,
        image_max_side=1333,
        auto_augment=None,
        rand_augment=(N,M),
        config=None
        )
    return train_generator

def create_test_generator():
    test_generator = csv_generator.CSVGenerator(
    test_annotations,
    classes,
    shuffle_groups=False,
    auto_augment=None,
    rand_augment=None,
    config=None)
    return test_generator
    
train_generator = create_train_generator(default_params['N'], default_params['M'])
test_generator = create_test_generator()

In [3]:
def create_model():
    model_path = Path('/data/students_home/fschipani/thesis/keras-retinanet/snapshots/resnet50_coco_best_v2.1.0.h5')
    model = models.backbone('resnet50').retinanet(num_classes=train_generator.num_classes())
    model.load_weights(model_path, by_name=True, skip_mismatch=True)
    model.compile(
        loss={
            'regression'    : smooth_l1(),
            'classification': focal()
        },
        optimizer=keras.optimizers.adam(lr=1e-5, clipnorm=0.001)
    )
    return model

In [4]:
def test_model(model, generator, score_threshold):
    result = evaluate(
        generator,
        models.convert_model(model, None),
        score_threshold=score_threshold,
        iou_threshold=0.5,
        max_detections=100,
        save_path=None
    )
    return result

test_results = test_model(create_model(), create_test_generator(), 0.3)

/data/students_home/fschipani/.local/lib/python3.7/site-packages/keras/engine/saving.py:1319: UserWarning: Skipping loading of weights for layer classification_submodel due to mismatch in shape ((3, 3, 256, 27) vs (720, 256, 3, 3)).
  weight_values[i].shape))
/data/students_home/fschipani/.local/lib/python3.7/site-packages/keras/engine/saving.py:1319: UserWarning: Skipping loading of weights for layer classification_submodel due to mismatch in shape ((27,) vs (720,)).
  weight_values[i].shape))


tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-22.627417, -11.313708,  22.627417,  11.313708],
       [-28.50876 , -14.25438 ,  28.50876 ,  14.25438 ],
       [-35.918785, -17.959393,  35.918785,  17.959393],
       [-16.      , -16.      ,  16.      ,  16.      ],
       [-20.158737, -20.158737,  20.158737,  20.158737],
       [-25.398417, -25.398417,  25.398417,  25.398417],
       [-11.313708, -22.627417,  11.313708,  22.627417],
       [-14.25438 , -28.50876 ,  14.25438 ,  28.50876 ],
       [-17.959393, -35.918785,  17.959393,  35.918785]], dtype=float32)> anchors
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-45.254833, -22.627417,  45.254833,  22.627417],
       [-57.01752 , -28.50876 ,  57.01752 ,  28.50876 ],
       [-71.83757 , -35.918785,  71.83757 ,  35.918785],
       [-32.      , -32.      ,  32.      ,  32.      ],
       [-40.317474, -40.317474,  40.317474,  40.317474],
       [-50.796833, -50.796833,  50.7

Running network: 100% (400 of 400) |#####| Elapsed Time: 0:00:54 Time:  0:00:54
Parsing annotations: 100% (400 of 400) |#| Elapsed Time: 0:00:00 Time:  0:00:00


In [5]:
test_results[2][0] #mAP for person

0.0

In [6]:
config = {
      "algorithm": "bayes",
      "parameters": {
          "N": {"type": "integer", "min": 1, "max": 3},
          "M": {"type": "integer", "min": 5, "max": 30},
      },
      "spec": {
          "metric": "mAP",
          "objective": "minimize",
      },
  }
opt = Optimizer(config, api_key=API_KEY, project_name="MSC-Thesis-PJ")


COMET WARNING: Passing Experiment through Optimizer constructor is deprecated; pass them to Optimizer.get_experiments or Optimizer.next
COMET INFO: COMET_OPTIMIZER_ID=fcdddabaa54a4ee79a15f93e94a131be
COMET INFO: Using optimizer config: {'algorithm': 'bayes', 'configSpaceSize': 'infinite', 'endTime': None, 'id': 'fcdddabaa54a4ee79a15f93e94a131be', 'lastUpdateTime': None, 'maxCombo': 0, 'name': 'fcdddabaa54a4ee79a15f93e94a131be', 'parameters': {'M': {'max': 30, 'min': 5, 'scalingType': 'uniform', 'type': 'integer'}, 'N': {'max': 3, 'min': 1, 'scalingType': 'uniform', 'type': 'integer'}}, 'predictor': None, 'spec': {'gridSize': 10, 'maxCombo': 0, 'metric': 'mAP', 'minSampleSize': 100, 'objective': 'minimize', 'retryAssignLimit': 0, 'retryLimit': 20, 'seed': 871936509}, 'startTime': 10543286431, 'state': {'sequence_i': 0, 'sequence_pid': None, 'sequence_retry': 0}, 'status': 'running', 'suggestion_count': 0, 'trials': 1, 'version': '1.0.24'}


In [ ]:
for experiment in opt.get_experiments():
    N = experiment.get_parameter('N')
    M = experiment.get_parameter('M')
    train_generator = create_train_generator(N, M)
    model = create_model()
    history = model.fit_generator(generator=train_generator, steps_per_epoch=10000, epochs=5, verbose=0)
    test_results = test_model(model, create_test_generator(), 0.3)
    experiment.log_metric("mAP", -test_results[2][0])
    

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/iskorini/msc-thesis-pj/8fa9683720c646cb872eaed8b22669b0

/data/students_home/fschipani/.local/lib/python3.7/site-packages/keras/engine/saving.py:1319: UserWarning: Skipping loading of weights for layer classification_submodel due to mismatch in shape ((3, 3, 256, 27) vs (720, 256, 3, 3)).
  weight_values[i].shape))
/data/students_home/fschipani/.local/lib/python3.7/site-packages/keras/engine/saving.py:1319: UserWarning: Skipping loading of weights for layer classification_submodel due to mismatch in shape ((27,) vs (720,)).
  weight_values[i].shape))
COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE
COMET INFO: Ignoring automatic log_parameter('do_validation') because 'keras:do_validation' is in COMET_LOGGING_PARAMETERS_IGNORE
COMET INFO: Ignoring automatic log_metric('batch_batch') because 'keras:batch_batch' is in COMET_LOGGING_METRICS_IGNORE
COMET INFO:

W1205 17:15:27.340494 140245286700800 def_function.py:474] 8 out of the last 11 calls to <function from_4D_image at 0x7f8d0a699510> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/beta/tutorials/eager/tf_function#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.
W1205 17:15:27.344323 140245286700800 def_function.py:474] 9 out of the last 11 calls to <function transform at 0x7f8d0a6a6a60> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please re

W1205 17:15:30.518346 140245286700800 def_function.py:474] 9 out of the last 11 calls to <function to_4D_image at 0x7f8d0a699378> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/beta/tutorials/eager/tf_function#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.
W1205 17:15:30.544586 140245286700800 def_function.py:474] 9 out of the last 11 calls to <function to_4D_image at 0x7f8d0a699378> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please re

W1205 17:15:39.474686 140245286700800 def_function.py:474] 11 out of the last 11 calls to <function transform at 0x7f8d0a6a6a60> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/beta/tutorials/eager/tf_function#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.
W1205 17:15:42.579505 140245286700800 def_function.py:474] 9 out of the last 11 calls to <function to_4D_image at 0x7f8d0a699378> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please ref

W1205 17:15:48.156200 140245286700800 def_function.py:474] 6 out of the last 11 calls to <function transform at 0x7f8d0a6a6a60> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/beta/tutorials/eager/tf_function#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.


...

W1205 17:18:17.033475 140235077826304 def_function.py:474] 11 out of the last 11 calls to <function to_4D_image at 0x7f8d0a699378> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/beta/tutorials/eager/tf_function#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.
W1205 17:18:17.073207 140235077826304 def_function.py:474] 11 out of the last 11 calls to <function from_4D_image at 0x7f8d0a699510> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Pleas

------------------------------------------------
tf.Tensor(0.4296875, shape=(), dtype=float32)
tf.Tensor(0.4734375, shape=(), dtype=float32)
tf.Tensor(333, shape=(), dtype=int32)
[<tf.Tensor: id=273059, shape=(), dtype=int32, numpy=74>, <tf.Tensor: id=273061, shape=(), dtype=int32, numpy=30>]
------------------------------------------------


W1205 17:18:21.366365 140235077826304 def_function.py:474] 11 out of the last 11 calls to <function to_4D_image at 0x7f8d0a699378> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/beta/tutorials/eager/tf_function#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.
W1205 17:18:21.391403 140235077826304 def_function.py:474] 11 out of the last 11 calls to <function to_4D_image at 0x7f8d0a699378> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please 

W1205 17:18:22.702345 140235077826304 def_function.py:474] 11 out of the last 11 calls to <function to_4D_image at 0x7f8d0a699378> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/beta/tutorials/eager/tf_function#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.
W1205 17:18:22.716195 140235077826304 def_function.py:474] 11 out of the last 11 calls to <function from_4D_image at 0x7f8d0a699510> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Pleas

------------------------------------------------
tf.Tensor(0.8828125, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(0, shape=(), dtype=int32)
[<tf.Tensor: id=285846, shape=(), dtype=int32, numpy=116>, <tf.Tensor: id=285848, shape=(), dtype=int32, numpy=47>]
------------------------------------------------


In [ ]:
#plt.figure()
#xs = [t['tid'] for t in trials.trials]
#ys = [-t['result']['mAP'] for t in trials.trials]
#plt.xlim(xs[0]-1, xs[-1]+1)
#plt.scatter(xs, ys, s=20, linewidth=0.01, alpha=0.75)
#plt.xlabel('Iteration', fontsize=16)
#plt.ylabel('Accuracy', fontsize=16)
#plt.show()